In [1]:
import numpy as np
import tkinter as tk
from keras.models import load_model
from PIL import  ImageOps, Image, ImageTk, ImageDraw

In [2]:

# Load your pre-trained model
model = load_model("mnist.h5")

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert("L")
    img = np.array(img)
    # Reshape to support our model input and normalize
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predicting the class
    res = model.predict(img)[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48), fg="black")
        self.classify_btn = tk.Button(
            self, text="Recognize", command=self.classify_handwriting
        )
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=tk.W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)

        self.image = Image.new("L", (300, 300), "white")
        self.draw = ImageDraw.Draw(self.image)

    def clear_all(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (300, 300), "white")
        self.draw = ImageDraw.Draw(self.image)

    def draw_lines(self, event):
        x1, y1 = (event.x - 1), (event.y - 1)
        x2, y2 = (event.x + 1), (event.y + 1)
        self.canvas.create_oval(x1, y1, x2, y2, fill="black")
        self.draw.line([x1, y1, x2, y2], fill="black", width=15)

    def classify_handwriting(self):
        # Resize the image to 28x28 pixels
        img = self.image.resize((28, 28))
        img = ImageOps.invert(img)
        digit, acc = predict_digit(img)
        self.label.configure(text=str(digit) + ", " + str(int(acc * 100)) + "%")
        
        img_tk = ImageTk.PhotoImage(ImageOps.invert(self.image))

        # Display the inverted image in a new window
        img_window = tk.Toplevel(self)
        img_window.title("Image sent to model")
        img_label = tk.Label(img_window, image=img_tk)
        img_label.pack()
        img_window.mainloop()

app = App()
app.mainloop()


1/1 [==============================] - 0s 34ms/step
